## TF-IDFのスコアを各店ごと計算


In [1]:
import pandas as pd
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# CSVファイルを読み込む
try:
    df = pd.read_csv('odaiba_reviews.csv')
except FileNotFoundError:
    print("エラー: 'odaiba_reviews.csv'が見つかりません。")
    df = pd.DataFrame()

# JanomeのTokenizerを準備
t = Tokenizer()

# ストップワード（頻出するが分析に不要な単語）
stop_words = ['こと', 'もの', 'それ', 'これ', 'さん', 'よう', 'ため', 'こちら', 'ところ', '感じ', '利用', '訪問', '注文', '提供', '料理', 'お店', '感じ', '雰囲気', '時間', '今回', '非常']

# ステップ1:店舗ごとの口コミを「分かち書き」する 
def tokenize(text):
    """テキストを形態素解析し、名詞・動詞・形容詞の原型を空白区切りで返す"""
    tokens = []
    for token in t.tokenize(text):
        # 品詞が名詞, 動詞, 形容詞のいずれかで、ストップワードに含まれない単語を抽出
        part_of_speech = token.part_of_speech.split(',')[0]
        if part_of_speech in ['名詞', '動詞', '形容詞'] and token.surface not in stop_words:
            # base_formは単語の原型（例：「食べました」→「食べる」）
            tokens.append(token.base_form)
    return ' '.join(tokens)

if not df.empty:
    # 店舗ごとに口コミを連結
    shop_docs = df.groupby('shop_name')['review_text'].apply(lambda x: ' '.join(x.dropna())).reset_index()
    
    # 各店舗の連結済み口コミに、分かち書き処理を適用
    print("各店舗の口コミを分かち書き中...")
    shop_docs['tokens'] = shop_docs['review_text'].apply(tokenize)
    
    # TF-IDFで分析するための「文書リスト」（コーパス）を作成
    corpus = shop_docs['tokens'].tolist()
    shop_names = shop_docs['shop_name'].tolist()

    # ステップ2：TF-IDF計算
    if corpus:
        print("TF-IDFスコアを計算中...")
        # TfidfVectorizerを初期化
        # min_df=2: 2店舗未満にしか出現しないレアすぎる単語は無視する
        vectorizer = TfidfVectorizer(min_df=2, max_df=0.95)
        
        # コーパスを元にTF-IDF行列を計算
        tfidf_matrix = vectorizer.fit_transform(corpus)
        
        # 単語のリスト（特徴量名）を取得
        feature_names = vectorizer.get_feature_names_out()

        # ステップ3：結果を店舗ごとに表示
        print("\n--- 店舗ごとの特徴語（TF-IDFスコア上位）---\n")
        
        for i, shop_name in enumerate(shop_names):
            print(f"【{shop_name}】")
            
            # i番目の店舗（文書）に対応するTF-IDFスコアの行を取得
            row = tfidf_matrix.toarray()[i]
            
            # スコアが高い順に単語のインデックスを並べ替え
            top_n_indices = row.argsort()[::-1][:7] # 上位7件
            
            # トップ7の単語とそのスコアを表示
            for index in top_n_indices:
                # スコアが0の単語は表示しない
                if row[index] > 0:
                    print(f"  - {feature_names[index]} (スコア: {row[index]:.3f})")
            print("-" * 20)
    else:
        print("分析対象のデータがありません。")

各店舗の口コミを分かち書き中...
TF-IDFスコアを計算中...

--- 店舗ごとの特徴語（TF-IDFスコア上位）---

【1129 by Ogawa】
  - ハンバーグ (スコア: 0.546)
  - 台場 (スコア: 0.295)
  - 景色 (スコア: 0.198)
  - デックス (スコア: 0.189)
  - ブリッジ (スコア: 0.163)
  - レインボー (スコア: 0.162)
  - 夜景 (スコア: 0.154)
--------------------
【82 築地店】
  - 築地 (スコア: 0.668)
  - パブ (スコア: 0.318)
  - 82 (スコア: 0.289)
  - hub (スコア: 0.231)
  - ビール (スコア: 0.123)
  - 銀座 (スコア: 0.118)
  - パイント (スコア: 0.110)
--------------------
【ABURI 百貫 有明ガーデン店】
  - 寿司 (スコア: 0.555)
  - 炙る (スコア: 0.519)
  - 回転 (スコア: 0.324)
  - ガーデン (スコア: 0.215)
  - 有明 (スコア: 0.186)
  - ネタ (スコア: 0.184)
  - 美味しい (スコア: 0.123)
--------------------
【ARIAKE ARENA CAFE】
  - アリーナ (スコア: 0.663)
  - ライブ (スコア: 0.271)
  - 有明 (スコア: 0.267)
  - トリュフ (スコア: 0.177)
  - 現金 (スコア: 0.123)
  - ホットドッグ (スコア: 0.114)
  - ariake (スコア: 0.112)
--------------------
【ARIAKE ARENA DINING MOON RIVER】
  - アリーナ (スコア: 0.509)
  - マルゲリータ (スコア: 0.318)
  - ライブ (スコア: 0.248)
  - 有明 (スコア: 0.240)
  - カフェ (スコア: 0.225)
  - イベント (スコア: 0.186)
  - ピザ (スコア: 0.184)
------------